In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import nltk
import tensorflow as tf
import os
import string
import re

In [2]:
data = pd.read_excel('sentence_past_present_future.xlsx')

In [3]:
y = data['Categorie']

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [5]:
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import one_hot
voc_size = 10000
input = []
for x in data['Sentences']:
    val = word_tokenize(x)
    one_rpr = one_hot(str(val),n=voc_size,lower=False,split=None)
    input.append(one_rpr)

In [6]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
sent_length = 15
embedded_docs = pad_sequences(input,padding='post',maxlen=sent_length)
dim = 10

In [7]:
model =Sequential()
model.add(Embedding(voc_size,dim,input_length=sent_length))
model.compile('adam','mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 10)            100000    
                                                                 
Total params: 100000 (390.62 KB)
Trainable params: 100000 (390.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
x = model.predict(embedded_docs)

87/87 [==============================] - 0s 1ms/step


In [9]:
rows = x.shape[0]
cols = x.shape[1] * x.shape[2]
x = x.reshape(rows,cols)

In [10]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.25,random_state=42)

In [11]:
from sklearn.tree import DecisionTreeClassifier

In [12]:
tree_clf = DecisionTreeClassifier() #max depth=5

In [13]:
tree_clf.fit(x_train,y_train)

DecisionTreeClassifier()

In [14]:
tree_clf.score(x_test,y_test)

0.7821067821067821

In [16]:
tree_clf.score(x_train,y_train)

1.0

In [17]:
from sklearn.metrics import accuracy_score , confusion_matrix , precision_score , recall_score , f1_score , roc_auc_score
def Evaluation_metrics(actual,pred):
    accuracy_sco = accuracy_score(actual,pred)
    confusion_mat = confusion_matrix(actual,pred)
    precision_sco = precision_score(actual,pred,average='macro')
    recall_sco = recall_score(actual,pred,average='macro')
    f1_sco = f1_score(actual,pred,average='macro')
    return accuracy_sco,confusion_mat,precision_sco,recall_sco,f1_sco

In [19]:
predicted_val = tree_clf.predict(x_test)

In [20]:
Evaluation_metrics(y_test,predicted_val)

(0.7821067821067821,
 array([[207,  15,  10],
        [ 24, 147,  60],
        [ 10,  32, 188]], dtype=int64),
 0.7817784303770828,
 0.7819987733406024,
 0.7791735932580498)